In [ ]:
# import sys
# sys.path.insert(0, 'driftpy/src/')
# import driftpy
# driftpy.__file__

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

# LOAD devnet environment

In [ ]:
import os
from driftpy.clearing_house import ClearingHouse
from driftpy.clearing_house_user import ClearingHouseUser

from arb_helpers import load_position_table, calculate_market_summary

os.environ['ANCHOR_WALLET'] = os.path.expanduser('~/.config/solana/<YOURWALLET>.json')

ENV = 'devnet'
drift_acct = await ClearingHouse.create_from_env(ENV)
drift_user = ClearingHouseUser(drift_acct, drift_acct.program.provider.wallet.public_key)

In [ ]:
from driftpy.constants.config import CONFIG

In [ ]:
# pd.DataFrame((await drift_user.get_user_positions_account()).positions)

In [ ]:
# market = await drift_user.clearing_house.get_market(0)
# (await drift_user.get_position_value(0))/1e6

# load user state info

In [ ]:
?drift_acct.initialize_user_account_and_deposit_collateral

In [ ]:
drift_user_acct = await drift_user.get_user_account()
balance = (drift_user_acct.collateral/1e6)
balance

In [ ]:
position_table_df = await load_position_table(drift_acct)
position_table_df

In [ ]:
QUOTE_PRECISION = 1e6
async def get_margin_account_info(drift_user):
    drift_user_acct = await drift_user.get_user_account()
    margin_info = {}
    margin_info['total_collateral'] = (await drift_user.get_total_collateral())/QUOTE_PRECISION
    margin_info['collateral'] = (drift_user_acct.collateral/QUOTE_PRECISION)
    margin_info['unrealised_pnl'] = (await drift_user.get_unrealised_pnl(0))/QUOTE_PRECISION
    # margin_info['margin_ratio'] = await drift_user.get_margin_ratio()
    margin_info['leverage'] = await drift_user.get_leverage()
    # margin_info['free_collateral'] = (await drift_user.get_free_collateral())/QUOTE_PRECISION
    return pd.Series(margin_info)

await get_margin_account_info(drift_user)

In [ ]:
from solana.publickey import PublicKey
from spl.token.instructions import get_associated_token_address

drift_state = await drift_acct.get_state_account()

USDC_AMOUNT = 1
USDC_AMOUNT_IN_FULL_PRECISION = int(USDC_AMOUNT * QUOTE_PRECISION)

# get user authority associated usdc account address
user_authority_usdc_account = get_associated_token_address( drift_acct.program.provider.wallet.public_key, 
                              PublicKey(CONFIG[ENV]['USDC_MINT_ADDRESS']))


# withdraw and deposit 1 USDC
await drift_acct.withdraw_collateral(USDC_AMOUNT_IN_FULL_PRECISION,
                                     user_authority_usdc_account
                                    )
await drift_acct.deposit_collateral(USDC_AMOUNT_IN_FULL_PRECISION, 
                                    user_authority_usdc_account
                                    )

# load predicted funding

In [ ]:
markets = await drift_acct.get_markets_account()
market_summary = await calculate_market_summary(markets)

In [ ]:
market_summary

In [ ]:
position_table_df

In [ ]:
# check number of active positions

if len(position_table_df.dropna())==5:
    print('account at maximum positions (5)')
else:
    print('account has room for more positions')


In [ ]:
# close smallest position (if less than $10)

smallest_position = position_table_df.sort_values('notional').iloc[0]
if smallest_position.notional < 10:
    await drift_acct.close_position(int(smallest_position.market_index))
else:
    print('not closing position over $10')

In [ ]:
# open a $1 LONG position in the most attractive markets for next funding

from driftpy.types import PositionDirection
market_to_trade = market_summary.sort_values('next_funding_rate(%)').iloc[0].market_index

await drift_acct.open_position(PositionDirection.LONG(), int(1 * 1e6), market_to_trade)

# use ftx hedge

In [ ]:
import json
import os

# construct secret ~/.ftx json file for api
FTX_SECRET_FF = os.path.expanduser('~/.ftx')
# ftx_secret = {
#     'apiKey' : '', 
#     'secret' : '', 
#     'headers': {
#         'FTX-SUBACCOUNT' : ''
#     }
# }
# with open('FTX_SECRET_FF', 'w') as f:
#     json.dump(ftx_secret, f)
    
ftx_secret = None
with open(FTX_SECRET_FF, 'r') as f:
    ftx_secret = json.load(f)

In [ ]:
# use ccxt library to interact with ftx
# !pip install ccxt

import ccxt
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

ftx = ccxt.ftx(ftx_secret)
pd.DataFrame(ftx.fetch_balance()['info']['result'])

In [ ]:
positions = pd.DataFrame(ftx.fetch_positions())
print(positions.columns)
positions[['symbol','notional',
           'unrealizedPnl','contracts',
           'markPrice', 'percentage', 'liquidationPrice',
           'side',
          ]]

In [ ]:
mytrades = pd.DataFrame(ftx.fetch_my_trades())
print(mytrades.columns)
mytrades = mytrades[['datetime', 'symbol', 'takerOrMaker', 'side', 'amount', 'price', 'cost', 'fee']]
mytrades.fee = mytrades.fee.apply(lambda x: x['cost'])
mytrades

In [ ]:
ftx.createLimitOrder('XXX-PERP', 'buy', 1, 100, params={'postOnly':True})

In [ ]:
pd.DataFrame(ftx.fetch_open_orders()).dropna(how='all',axis=1)

In [ ]:
ftx.cancelAllOrders()